## to do
- improve context extractor
- add vector db
- test different models

In [ ]:
# Install essential libraries
!pip install -q wikipedia duckduckgo-search transformers accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline, AutoModel
import wikipedia
from duckduckgo_search import DDGS
import re
import torch
import logging
import os
import pandas as pd
import gc
import psutil

In [ ]:
# login for gated models
from huggingface_hub import login
login()

Note: Upload your data

In [ ]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

In [ ]:
def create_question(row):
    """
    Creating the right format for questions
    """
    q = row['prompt']
    for col in ['A','B','C','D','E']:
        q += f'\n {col}) {row[col]}'
    return q


train_data['question'] = train_data.apply(create_question, axis=1)
train_data['question+answer'] = train_data['question'] + f'\n Answer: ' + train_data['answer']
test_data['question'] = test_data.apply(create_question, axis=1)

In [ ]:
print("Train data sample:")
for i in train_data['question'].sample().values:
    print(i, end='\n-------------\n')
print('Test data sample:')
for i in test_data['question'].sample().values:
    print(i, end='\n-------------\n')

Train data sample:
What is the main factor that determines the occurrence of each type of supernova?
 A) The star's distance from Earth
 B) The star's age
 C) The star's temperature
 D) The star's luminosity
 E) The progenitor star's metallicity
-------------
Test data sample:
What is a Schwarzschild black hole?
 A) A black hole that has mass but neither electric charge nor angular momentum, and is not spherically symmetric, according to Birkhoff's theorem.
 B) A black hole that has mass, electric charge, and angular momentum, and is spherically symmetric, according to Birkhoff's theorem.
 C) A black hole that has mass but neither electric charge nor angular momentum, and is spherically symmetric, according to Birkhoff's theorem.
 D) A black hole that has neither mass nor electric charge nor angular momentum, and is not spherically symmetric, according to Birkhoff's theorem.
 E) A black hole that has mass, electric charge, and angular momentum, and is not spherically symmetric, accordi

In [ ]:
train_data['prompt'].sample(10).values

array(['What is the main factor that determines the occurrence of each type of supernova?',
       "What is the 'reactive Leidenfrost effect' observed in non-volatile materials?",
       'What is accelerator-based light-ion fusion?',
       'What is the interpretation of supersymmetry in stochastic supersymmetric theory?',
       'What is a crossover experiment?',
       'What is the relationship between chemical potential and quarks/antiquarks?',
       "What is the revised view of the atmosphere's nature based on the time-varying multistability that is associated with the modulation of large-scale processes and aggregated feedback of small-scale processes?",
       'What is water hammer?',
       'What is the Landau-Lifshitz-Gilbert equation used for in physics?',
       'How many crystallographic point groups are there in three-dimensional space?'],
      dtype=object)

In [ ]:
train_data['answer'].value_counts()

,count
answer,
B,13
E,12
C,11
A,7
D,7


In [ ]:
# Set environment variables and configure logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging
logging.basicConfig(level=logging.ERROR)

def setup_device():
    """Set up and return the appropriate device for model inference."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    if device == "cuda":
        try:
            gc.collect()
            torch.cuda.empty_cache()
            print("CUDA cache cleared successfully")
        except Exception as e:
            print(f"Error clearing CUDA cache: {e}")

    return device

In [ ]:
def load_model_and_tokenizer(model_id, device, model_type=None):
    """
    Load and return the model and tokenizer with support for different architectures.

    Args:
        model_id (str): Hugging Face model identifier.
        device (str): 'cuda' or 'cpu'.
        model_type (str, optional): One of 'seq2seq', 'causal', or None for auto-detect.

    Returns:
        tuple: (model, tokenizer)
    """
    print(f"Loading model: {model_id}")

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        print("Tokenizer loaded successfully")

        # Auto-detect model type if not specified
        if model_type is None:
            if any(x in model_id.lower() for x in ['t5', 'bart', 'pegasus', 'mt5', 'mbart']):
                model_type = 'seq2seq'
            elif any(x in model_id.lower() for x in ['gpt', 'bloom', 'llama', 'falcon', 'mistral']):
                model_type = 'causal'
            else:
                model_type = 'auto'
            print(f"Auto-detected model type: {model_type}")

        # Setup device kwargs
        cuda_kwargs = {}
        if device == "cuda" and torch.cuda.is_available():
            print(f"Using CUDA with {torch.cuda.device_count()} GPU(s)")
            cuda_kwargs = {
                "device_map": "auto",
                "torch_dtype": torch.float16
            }

        # Load appropriate model type
        if model_type == 'seq2seq':
            model = AutoModelForSeq2SeqLM.from_pretrained(model_id, **cuda_kwargs)
        elif model_type == 'causal':
            model = AutoModelForCausalLM.from_pretrained(model_id, **cuda_kwargs)
        elif model_type == 'auto':
            try:
                model = AutoModelForSeq2SeqLM.from_pretrained(model_id, **cuda_kwargs)
            except Exception:
                try:
                    model = AutoModelForCausalLM.from_pretrained(model_id, **cuda_kwargs)
                except Exception:
                    model = AutoModel.from_pretrained(model_id, **cuda_kwargs)
        else:
            raise ValueError(f"Unsupported model_type: {model_type}")

        print(f"Model loaded successfully as {model.__class__.__name__}")
        return model, tokenizer

    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        raise


## extract concept from question

In [ ]:
def create_concept_extractor(model, tokenizer):
    """Create a pipeline for extracting the main concept from a question."""
    try:
        concept_extractor = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer
        )
        return concept_extractor
    except Exception as e:
        print(f"Error creating concept extractor: {e}")
        raise

def extract_main_concept(concept_extractor, question):
    """Extract the main concept from a question."""
    prompt = f"""
    Identify the main concept or topic from the following scientific question. Return only a short phrase that represents the core idea, such as the name of a scientific concept, physical law, equation, or device.

    Question: {question}

    Main concept:
    """
    response = concept_extractor(prompt, max_length=256, do_sample=False)[0]['generated_text']
    return response.strip()


In [ ]:
# Test

# device = setup_device()

# models = [
#     # 'google/flan-t5-xl',
#     # 'google/flan-t5-large',
#     'google/flan-t5-base',
#     # 'gpt2'
#      ]

# questions = train_data.sample(5, random_state=42).iloc[:,0]

# for model_id in models:
#     print(f"Testing model: {model_id}")
#     model, tokenizer = load_model_and_tokenizer(model_id, device)
#     concept_extractor = create_concept_extractor(model, tokenizer)

#     for question in questions:
#       concept = extract_main_concept(concept_extractor, question)

#       print("Question: ", question)
#       print("Main concept: ", concept)
#       print("-" * 50)

- best model for extracting concept: google/flan-t5-xl and google/flan-t5-large are both good
- I chose flan-t5-large becuase it's faster


## retrive context

In [ ]:
def get_wikipedia_summary(query, max_chars=2000):
    """Retrieve a summary from Wikipedia for the given query."""
    try:
        page = wikipedia.page(query)
    except:
        try:
            title = wikipedia.search(query)[0]
            page = wikipedia.page(title)
        except Exception as e:
            print(f"Wikipedia retrieval failed for query '{query}': {e}")
            return ""
    return page.content[:max_chars]

def get_duckduckgo_snippets(query, max_results=5):
    """Retrieve snippets from DuckDuckGo for the given query."""
    try:
        text = ""
        with DDGS() as ddgs:
            results = ddgs.text(query, region='wt-wt', safesearch='Moderate', max_results=max_results)
            for r in results:
                if 'body' in r:
                    text += r['body'] + "\n"
        return text.strip()
    except Exception as e:
        print(f"DuckDuckGo search failed for query '{query}': {e}")
        return ""

def retrieve_context(question, concept=None, use_ddg=False, wiki_chars=2000, ddg_results=5):
    """Retrieve context information from Wikipedia and optionally DuckDuckGo."""
    query = concept if concept else question
    wiki_text = get_wikipedia_summary(query, max_chars=wiki_chars)

    ddg_text = ""
    if use_ddg:
        ddg_text = get_duckduckgo_snippets(query, max_results=ddg_results)

    context = f"\n[Wikipedia]\n{wiki_text}"
    if use_ddg and ddg_text:
        context += f"\n\n[Web Search]\n{ddg_text}"

    return context.strip()

In [ ]:
# Test

# device = setup_device()
# print(device)

# model, tokenizer = load_model_and_tokenizer('google/flan-t5-large', device)
# concept_extractor = create_concept_extractor(model, tokenizer)

# question = train_data.iloc[0, 0]
# concept = extract_main_concept(concept_extractor, question)
# context = retrieve_context(question, concept, use_ddg=True)

# print("Question: ", question)
# print("Main concept: ", concept)
# print("Context: ", context)

## RAG

In [ ]:
def create_answer_selector(model, tokenizer):
    """Create a pipeline for selecting the answer option."""
    try:
        answer_selector = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=4,
            do_sample=False
        )
        return answer_selector
    except Exception as e:
        print(f"Error creating answer selector: {e}")
        raise

def create_prompt(questionـoptions, context, examples):
    """Create a prompt for the model to select an answer."""
    example_text = ''
    for ex in examples:
        example_text += str(ex).strip() + '\n\n'

    prompt = f"""
You are an expert AI assistant for answering scientific questions using external reference material.

### Context:
{context}

### Examples:
{example_text}

### Question:
{questionـoptions}

Based on the context above, choose the most accurate answer. Output exactly one letter (A, B, C, D, or E) with no additional text, punctuation, or explanation.
### Answer:
    """
    return prompt.strip()

def extract_letter(response):
    """Extract the answer letter from the model's response."""
    response = response.replace('answer', '')
    match = re.search(r"[A-E]", response)
    return match.group(0) if match else None


In [ ]:
# Test


# device = setup_device()
# print(device)

# model, tokenizer = load_model_and_tokenizer('google/flan-t5-large', device)
# concept_extractor = create_concept_extractor(model, tokenizer)

# model, tokenizer = load_model_and_tokenizer('google/flan-t5-xl', device)
# answer_selector = create_answer_selector(model, tokenizer)

# question = train_data.iloc[0, 0]
# questionـoptions = train_data.iloc[0, 7]
# answer = train_data.iloc[0, 6]
# concept = extract_main_concept(concept_extractor, question)
# context = retrieve_context(question, concept, use_ddg=True)
# examples = train_data.sample(2, random_state=42)['question+answer'].values
# prompt = create_prompt(questionـoptions, context, examples)

# response = answer_selector(prompt)[0]['generated_text']

# print("Question: ", question, end='\n------\n')
# print("Main concept: ", concept, end='\n------\n')
# print("Context: ", context, end='\n------\n')
# print("Prompt: ", prompt, end='\n------\n')
# print('Result:', response, end='\n------\n')
# print('Answer:', answer)

In [ ]:
# Main ScienceQA class that uses all the above functions
class ScienceQA:
    def __init__(self, model_id_ce, model_id_as, train_data=None):
        self.model_id_ce = model_id_ce
        self.model_id_as = model_id_as
        self.train_data = train_data

        # Setup
        self.device = setup_device()
        if model_id_ce == model_id_as:
          self.model_ce, self.tokenizer_ce = load_model_and_tokenizer(model_id_ce, self.device)
          self.model_as, self.tokenizer_as = load_model_and_tokenizer(model_id_as, self.device)

          # Create pipelines
          self.concept_extractor = create_concept_extractor(self.model_ce, self.tokenizer_ce)
          self.answer_selector = create_answer_selector(self.model_as, self.tokenizer_as)

        else:
          self.model_ce, self.tokenizer_ce = load_model_and_tokenizer(model_id_ce, self.device)
          self.model_as, self.tokenizer_as = load_model_and_tokenizer(model_id_as, self.device)

          # Create pipelines
          self.concept_extractor = create_concept_extractor(self.model_ce, self.tokenizer_ce)
          self.answer_selector = create_answer_selector(self.model_as, self.tokenizer_as)

        print("ScienceQA initialization complete")

    def process_question(self, question, question_options, examples, use_ddg=False):
        """Process a single question and return the predicted answer."""
        # Step 1: Extract main concept
        main_concept = extract_main_concept(self.concept_extractor, question)
        print(f"Extracted concept: {main_concept}")

        # Step 2: Retrieve context
        context = retrieve_context(question, main_concept, use_ddg)

        # Step 3: Create prompt and get answer
        prompt = create_prompt(question_options, context, examples)
        response = self.answer_selector(prompt)[0]['generated_text']

        pred = extract_letter(response)

        return pred, main_concept, context

    def test_sample(self, sample_idx=0):
        """Test the model with a sample from the training data."""
        if self.train_data is None:
            print("No training data provided")
            return

        row = self.train_data.iloc[sample_idx]
        question = row['prompt']
        question_options = row['question']
        examples = self.train_data.sample(2, random_state=6)['question+answer'].values

        pred, concept, _ = self.process_question(question, question_options, examples)

        print("Question:", question)
        print("Extracted concept:", concept)
        print("Correct answer:", row['answer'])
        print("Predicted answer:", pred)
        print("Result:", "Correct" if pred == row['answer'] else "Incorrect")

    def evaluate(self, test_data=None, print_results=False):
        """Evaluate the model on test data or training data."""
        data = test_data if test_data is not None else self.train_data
        if data is None:
            print("No data provided for evaluation")
            return

        correct = 0
        total_questions = len(data)
        results = []

        for idx, row in data.iterrows():
            question = row["prompt"]
            question_options = row["question"]
            examples = self.train_data.sample(2, random_state=25)["question+answer"].values

            pred, concept, _ = self.process_question(question, question_options, examples)
            results.append(pred)

            if print_results:
                print(f"Question #{idx}: {question}")
                print(f"Concept: {concept}")
                if "answer" in row:
                    print("Correct answer:", row["answer"])
                print("Predicted answer:", pred)
                if "answer" in row:
                    print("Result:", "Correct" if pred == row["answer"] else "Incorrect")
                print("-" * 50)

            if "answer" in row and pred == row["answer"]:
                correct += 1

        if "answer" in data.columns:
            accuracy = correct / total_questions
            print(f"Accuracy: {correct}/{total_questions} = {accuracy:.2%}")

        return results



- google/flan-t5-large, google/flan-t5-xl Accuracy: 35/50 : 70.00%
- google/flan-t5-xl, google/flan-t5-xl Accuracy: 36/50 : 72.00%
- google/flan-t5-xl", facebook/opt-1.3b Accuracy: 7/50 : 14.00%
- google/flan-t5-xl", MBZUAI/LaMini-Flan-T5-783M Accuracy: 19/50 : 38.00%

In [ ]:
rag = ScienceQA(model_id_ce="google/flan-t5-xl", model_id_as="google/flan-t5-xl", train_data=train_data)  # Replace with your model ID

# Test with a single example
rag.test_sample()

# Evaluate on train data
rag.evaluate(train_data, print_results=False)

Using device: cuda
CUDA cache cleared successfully
Loading model: google/flan-t5-xl
Tokenizer loaded successfully
Auto-detected model type: seq2seq
Using CUDA with 1 GPU(s)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully as T5ForConditionalGeneration
Loading model: google/flan-t5-xl
Tokenizer loaded successfully
Auto-detected model type: seq2seq
Using CUDA with 1 GPU(s)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0


...

Token indices sequence length is longer than the specified maximum sequence length for this model (937 > 512). Running this sequence through the model will result in indexing errors


Question: What is the main sequence in astronomy?
Extracted concept: main sequence
Correct answer: C
Predicted answer: C
Result: Correct
Extracted concept: main sequence
Extracted concept: ultraviolet catastrophe
Extracted concept: Carnot engine
Extracted concept: accelerator-based light-ion fusion
Extracted concept: coffee ring
Extracted concept: water hammer
Extracted concept: memristor
Extracted concept: electronic entropy
Wikipedia retrieval failed for query 'electronic entropy': Page id "electronic entry" does not match any pages. Try another id!
Extracted concept: illuminance
Extracted concept: classical mechanics
Extracted concept: Peierls bracket
Extracted concept: cosmic inflation
Extracted concept: linear frame dragging
Extracted concept: Ozma Problem
Extracted concept: fatty-acyl disorder
Extracted concept: Coordinated Universal Time
Extracted concept: planetary system
Extracted concept: quarks


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Extracted concept: molecular cloud
Extracted concept: Maxwell's Demon
Extracted concept: crystallinity
Extracted concept: shower-curtain effect
Extracted concept: crossover experiment
Extracted concept: supersymmetry
Extracted concept: Young's double-slit interferometer
Extracted concept: multistability
Extracted concept: regular polytope
Extracted concept: isophotal diameter
Extracted concept: order parameter
Extracted concept: supernova
Extracted concept: axioms
Extracted concept: reactive Leidenfrost effect
Extracted concept: base unit of time
Extracted concept: AT-cut quartz crystals
Extracted concept: Class L dwarfs
Extracted concept: Landau-Lifshitz-Gilbert equation
Extracted concept: pulmonary circulation system
Extracted concept: Environmental Science Center
Extracted concept: Modified Newtonian Dynamics
Extracted concept: probability density function
Extracted concept: quantum mechanics
Extracted concept: excess base metal
Extracted concept: spin quantum number
Extracted conce

['C',
 'E',
 'A',
 'A',
 'B',
 'E',
 'D',
 'E',
 'C',
 'A',
 'A',
 'A',
 'D',
 'C',
 'C',
 'C',
 'C',
 'A',
 'B',
 'C',
 'D',
 'E',
 'B',
 'E',
 'C',
 'B',
 'C',
 'E',
 'D',
 'B',
 'A',
 'A',
 'B',
 'E',
 'B',
 'C',
 'C',
 'D',
 'C',
 'E',
 'E',
 'A',
 'E',
 'E',
 'B',
 'C',
 'C',
 'E',
 'C',
 'D']

In [ ]:
results = rag.evaluate(test_data, print_results=False)
test_data['answer'] = results
test_data.drop(columns=['question']).to_csv('result.csv', index=False)